<a href="https://colab.research.google.com/github/nic-instruction/IT-211/blob/main/Weekly_Topics/Week_7_Topics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Learning Objectives

* JUnit 5 testing of classes and methods
* Objects and Classes basics
* Using and defining classes
* Mutators, accessors and private helper methods
* Constructors
* Defining the main() in custom defined classes


# Pacing

At this point, the book starts to split into chapters that cover very specific topics, so we'll be doing a couple at a time.

This week, we'll cover 'mutable objects', 'immutable objects', and 'designing classes'. Both mutable and immutable objects are fairly straight forward, while 'Designing Classes' is a little more complex and depends on knowing what objects are and do. Other intro books group these together and do not go into depth about objects, however our book has good reason for looking at objects in depth, and you'll need to know about them for the code that follows.

Similarly, next week, we'll be covering 'arrays of objects' and 'objects of arrays', which we'll need for 'extending classes' (inheritance).

After that, we'll be doing 1 chapter a week until the end of the quarter.

# Junit Testing (Class & Method)

* We will be using JUnit 5 (the latest version of JUnit), much of the documentation out there is still for JUnit 4, so here's a handy translation table: https://howtodoinjava.com/junit5/junit-5-vs-junit-4/
* A very good tutorial that includes some theory of testing and explains concepts like 'coverage', it's a little long, but we'll take a look in class: https://www.vogella.com/tutorials/JUnit/article.html
* We won't test them here, but mocks are a good thing to know about when dev'ing: https://www.vogella.com/tutorials/Mockito/article.html
* You can write what's called a "test bench" which will thouroughly test all Classes.  The goal is 100% code coverage and it should include edge cases as well as test cases.
* At some point, you'll want to do 'intigration testing' which means testing your classes all together in such a way that you can make sure their intigration is working propperly: https://phauer.com/2019/focus-integration-tests-mock-based-tests/#integration-tests
* Good reference for clear JUnit testing (note, any company you work for will likely have their own requirements for how tests are written, ask about them up front): https://phauer.com/2019/modern-best-practices-testing-java/


In [ ]:
%%writefile FlowOfExecution.java
class MyMethods{
    public static int doubleMe(int b) {
        return (b + b);
    }
        
    public static String doubleMe(String b) {
        return (b + " " + b); // Note, this will concatonate the strings together with a space between them.
    }
        
    public static double doubleMe(double b) {
        return (b + b);
    }
        
    public static long doubleMe(long b) {
        return (b + b);
    }
}

public class FlowOfExecution{
    public static void main(String [] args) {
        int intNum        = 5;
        String stringWord = "NOW";
        double doubleNum  = 10.404;

        System.out.println("Passing already declared variables to MyMethods.doubleMe()");
        System.out.println(MyMethods.doubleMe(intNum));
        System.out.println(MyMethods.doubleMe(stringWord));
        System.out.println(MyMethods.doubleMe(doubleNum) + "\n");
    }
}

In [ ]:
!javac FlowOfExecution.java
!java FlowOfExecution

# JUnit 5 test of two methods

I left a few methods untested, and neither class has actually had an instance created yet because I've only tested public static methods so far.  We'll want to build both classes and test all our methods, but for now, here's an example: 

In [ ]:
import static org.junit.Assert.assertEquals;

import org.junit.jupiter.api.DisplayName;
import org.junit.jupiter.api.Test;

class FlowOfExecutionTest {

	@Test
	@DisplayName("Test overloaded method dobuleMe() for accurate string value.") 
    public void testDoubleMeString() {
		
		String expected = "test test";
		String actual = MyMethods.doubleMe("test");   // Given & When because public static void
		
		assertEquals(expected, actual);
		
		
	}
	
    @Test
    @DisplayName("Test overloaded method doubleMe() for accurate long value.")
    public void testDoubleMeLong() {
    	
    	long expected = 20000000L;
    	long actual   = MyMethods.doubleMe(10000000L);
    	
    	assertEquals(expected, actual);
    }
	

# Junit Testing Output

As we've seen, testing a return value is still complex, but relatively straight forward.  What do you do if you want to test a method that DOES something?  In other words, a method like `System.out.println` that has no return value (is `void`) but, for example, prints something to standard out.

Well, the answer depends on what, exactly, the method does, but let's take an example of one that prints something to standard out.  How do you capture that output?

Let's look at an example:

In [ ]:
%%writefile RecursiveMethod.java

public class RecursiveMethod {
    public static void main(String [] args) {
        breakDown("print me!");
        
    }

    // Example recursive method that returns nothing, but breaks down a string into chars
    // and prints each char.  
    
    public static void breakDown(String a) {
        int length = a.length();
        if (length == 0) {
            return;
        } else {
            System.out.println(a.charAt(0));
            a = a.substring(1, length);
            breakDown(a);
        }
    }

}

In [ ]:
!javac RecursiveMethod.java
!java RecursiveMethod

So, as you see, it will print the string it was given to standard out, letter by letter, with a new line in between each letter.  How can we test this when we don't have access to standard out?

The answer is to hook the print stream up to something that isn't standard out, and has a method for converting what it has stored/ captured into a string.  we do this by creating a `PrintStream` object and inputting our byteArrayOS variable which is a new object of the ByteArrayOutputStream class.  We do that, run our method, so that it gets handled by the ByteArrayOutputStream object and then use the ByteArrayOutputStream method `.toString` to turn it into a string variable, we've chosen to call 'result'.  

See below:

In [ ]:
%%writefile TestRecursiveMethodJUnit5.java
import static org.junit.Assert.assertEquals;
import org.junit.jupiter.api.Test;
import org.junit.jupiter.api.DisplayName;

import java.io.ByteArrayOutputStream;
import java.io.PrintStream;


class TestRecursiveMethodJUnit5 {

	@Test
  @DisplayName("Any String Entered Should Be Output 1 Letter Per Line")

	public void testBreakDown () {
		ByteArrayOutputStream byteArrayOS = new ByteArrayOutputStream();
		PrintStream ps = new PrintStream(byteArrayOS);
		PrintStream old = System.out;            // don't lose System.out, save it.  
		System.setOut(ps);                       // use your stream instead of System Out.
		
	    RecursiveMethod.breakDown("print me!");  // Both Given and When
		                                         // Normally we would need to create an instance, but it's public static
		                                         // so we don't!
		                                      
	    System.out.flush();                      // Clean things up, flush it
	    System.setOut(old);                      // put System.out back
	    String result = byteArrayOS.toString();  // convert the stream to a string and store it in the string 'result'
	    
	    String expected = "p\n"
				+ "r\n"
				+ "i\n"
				+ "n\n"
				+ "t\n"
				+ " \n"
				+ "m\n"
				+ "e\n"
				+ "!" 
				+ "\n" ;
		
		assertEquals(expected, result);
	}

}

This is not something you can test from the command line at this point.  If you really wanted to, you could create a 'test runner class': https://www.tutorialspoint.com/junit/junit_executing_tests.htm

But I recommend you wait for Gradle which we'll cover in a couple of weeks.  It will make command-line testing intuitive and easy.

That said, it's easy to run a JUnit test in eclipse and most other IDE's, so we'll be doing that in class.

# Immutable Objects

You've already been using them!  `String` is the most popular immutable object, similarly, BigInteger, and all the wrapper classes for primitives:  Boolean, Character, Double, and Long.  Note, the wrapper classes are really different from the primitives but have methods available unlike primitives.

Because they are objects you must use .equals() to compare numbers in them, just like with String.  == will only tell you if they are pointed to the same object in memory, not whether or not they have the same value.

Primitives: int, double, char, and boolean


# Mutable Objects

You've already been using them!  Arrays are mutable objects, though their length is not changeable, their contents can change no problem!  Also, StringBuilder, which is a handy alternative to setting a String equal to itself plus a concatenation, which is messy and relies on garbage collection.  

java.util.Date is a mutable object.  Good examples: https://examples.javacodegeeks.com/java-util-date/

You can access their attributes (internal values) using dot notation.